In [1]:
import keras
from keras.preprocessing import sequence
import numpy as np
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow import set_random_seed

# compute accuracy without mekeras method
from sklearn.metrics import accuracy_score

# fix random seed for reproducibility
import random

/home/jayati/anaconda3/envs/python2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)


In [3]:
import numpy as np
from keras.datasets import mnist

# read training data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_trainset = X_train.reshape(60000, 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32') / 255

y_trainset = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

X_trainset = X_trainset[:6000]
y_trainset = y_trainset[:6000]

#test data
test_size = 100
idx = np.random.choice(range(len(X_test)), size=test_size, replace=False)
X_test = X_test[idx]
y_test = y_test[idx]

train_size = 2000
idx = np.random.choice(range(len(X_trainset)), size=train_size, replace=False)

batch_size = X_train.shape[0]/5
num_epochs = 1
step = 200

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier


def mymodel():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    
    return model

In [5]:
from modAL.models import ActiveLearner
classifier = KerasClassifier(mymodel)
# initialize ActiveLearner
from modAL.models import ActiveLearner

# initialize ActiveLearner
learner = ActiveLearner(estimator=classifier,X_training=X_trainset[idx],batch_size=batch_size,
                        y_training=y_trainset[idx], epochs=num_epochs,verbose=1, 
                        callbacks=None, validation_split=0.0, 
                        validation_data=None, shuffle=True, 
                        class_weight=None, sample_weight=None, 
                        initial_epoch=0, steps_per_epoch=None,
                        validation_steps=None)

Epoch 1/1
2000/2000 [==============================] - 18s 9ms/step - loss: 2.2999 - acc: 0.1235


In [6]:
#print( "AC - acc:  " + str(learner.score(X_test, y_test)))
old_acc = learner.score(X_test, y_test)
print( "AC - acc:  " + str(old_acc))

100/100 [==============================] - 0s 4ms/step
AC - acc:  0.63


In [7]:
pool_idx = np.setdiff1d(range(len(X_trainset)),idx)

In [8]:
X_pool = X_trainset
y_pool = y_trainset

In [9]:
# the active learning loop
n_queries = len(pool_idx)/step
performance_history =[]

for idx in range(n_queries):
    print('Query no. %d' % (idx + 1))
    query_idx, query_instance = learner.query(X_pool[pool_idx], n_instances=step, verbose=0)
    learner.teach(
                  X=X_pool[query_idx], 
                  y=y_pool[query_idx], 
                  only_new=False,
                  verbose=1,epochs=num_epochs,callbacks=None, validation_split=0.0, 
                    validation_data=None, shuffle=True, 
                    class_weight=None, sample_weight=None, 
                    initial_epoch=0, steps_per_epoch=None,
                    validation_steps=None
                 )
    # remove queried instance from pool
    pool_idx = np.setdiff1d(pool_idx, query_idx)
    
    model_accuracy = learner.score(X_test, y_test)
    print('Accuracy after query {n}: {acc:0.4f}'.format(n=idx + 1, acc=model_accuracy))
    
    # Save our model's performance for plotting.
    performance_history.append(model_accuracy)

Query no. 1
Epoch 1/1
100/100 [==============================] - 1s 6ms/step
Accuracy after query 1: 0.9000
Query no. 2
Epoch 1/1
100/100 [==============================] - 1s 7ms/step
Accuracy after query 2: 0.9300
Query no. 3
Epoch 1/1
100/100 [==============================] - 1s 6ms/step
Accuracy after query 3: 0.8700
Query no. 4
Epoch 1/1
100/100 [==============================] - 1s 7ms/step
Accuracy after query 4: 0.9100
Query no. 5
Epoch 1/1
100/100 [==============================] - 1s 8ms/step
Accuracy after query 5: 0.8900
Query no. 6
Epoch 1/1
100/100 [==============================] - 1s 7ms/step
Accuracy after query 6: 0.9500
Query no. 7
Epoch 1/1
100/100 [==============================] - 1s 9ms/step
Accuracy after query 7: 0.9300
Query no. 8
Epoch 1/1
100/100 [==============================] - 1s 11ms/step
Accuracy after query 8: 0.9100
Query no. 9
Epoch 1/1
100/100 [==============================] - 1s 9ms/step
Accuracy after query 9: 0.9500
Query no. 10
Epoch 1/1
100/

In [10]:
#Add the starting accuracy to list
#acc_list =[]
#acc_list = performance_history
#acc_list.insert(0,old_acc)

In [11]:
performance_history = [old_acc]+performance_history

In [12]:
from matplotlib import pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,8))
sns.set_style("whitegrid")
sns.pointplot(x=range(train_size,len(X_pool)+1,step), y=performance_history, markers="o",linestyles="-")
plt.grid(True)

In [13]:
performance_history

[0.63,
 0.9,
 0.93,
 0.87,
 0.91,
 0.89,
 0.95,
 0.93,
 0.91,
 0.95,
 0.96,
 0.92,
 0.93,
 0.91,
 0.91,
 0.94,
 0.93,
 0.96,
 0.95,
 0.97,
 0.96]